# Get some data to play with

## FashionMNIST

In [1]:
import torchvision
import torch
import os

In [2]:
ROOT = os.getcwd()
DATA = os.path.join(ROOT,'data')

In [3]:
fashionMNIST_data = torchvision.datasets.FashionMNIST(DATA,download=True)

## Preprocessing

In [4]:
from torchvision import transforms

In [5]:
batch_size = 16

In [6]:
train_transforms = transforms.Compose([#transforms.Resize(size = 224),
                                       #transforms.CenterCrop(224),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                       ])
fashionMNIST_data.transform = train_transforms
data_loader = torch.utils.data.DataLoader(fashionMNIST_data,
                                          batch_size=batch_size,
                                          shuffle=True)

In [7]:
trainiter = iter(data_loader)
imgs, labels = trainiter.next()

In [8]:
example = imgs[0]
print(example.size())
transforms.ToPILImage()(example)

torch.Size([1, 28, 28])


## Network

In [9]:
from torch.autograd import Variable
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F

In [10]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(1, 16, 3, stride=3, padding=1),  # b, 16, 10, 10
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),  # b, 16, 5, 5
            nn.Conv2d(16, 8, 3, stride=2, padding=1),  # b, 8, 3, 3
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)  # b, 8, 2, 2
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, 3, stride=2),  # b, 16, 5, 5
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 5, stride=3, padding=1),  # b, 8, 15, 15
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 1, 2, stride=2, padding=1),  # b, 1, 28, 28
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

mynet = AutoEncoder()
mynet

AutoEncoder(
  (encoder): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(3, 3), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1), ceil_mode=False)
    (3): Conv2d(16, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=(2, 2), stride=(1, 1), dilation=(1, 1), ceil_mode=False)
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(8, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU(inplace)
    (2): ConvTranspose2d(16, 8, kernel_size=(5, 5), stride=(3, 3), padding=(1, 1))
    (3): ReLU(inplace)
    (4): ConvTranspose2d(8, 1, kernel_size=(2, 2), stride=(2, 2), padding=(1, 1))
    (5): Tanh()
  )
)

## Hyperparameters

In [11]:
learning_rate = 1e-3
num_epochs = 10
weight_decay = 1e-5

In [12]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(mynet.parameters(),
                             lr=learning_rate,
                             weight_decay=weight_decay
                            )

## Training

In [13]:
from torchvision.utils import save_image

def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x

In [14]:
losses = []
for epoch in range(num_epochs):
    for data in data_loader:
        img, _ = data
        img = Variable(img)
        # ===================forward=====================
        output = mynet(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}'
          .format(epoch+1, num_epochs, loss.data[0]))
    losses.append(loss.data[0])
    if epoch % 5 == 0:
        pic = to_img(output.cpu().data)
        save_image(pic, os.path.join(DATA,'image_{}.png'.format(epoch)))

epoch [1/10], loss:0.0840
epoch [2/10], loss:0.0725
epoch [3/10], loss:0.0938
epoch [4/10], loss:0.0861
epoch [5/10], loss:0.0791
epoch [6/10], loss:0.0702
epoch [7/10], loss:0.0770
epoch [8/10], loss:0.0961
epoch [9/10], loss:0.0708
epoch [10/10], loss:0.0897
